# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
print(filepath)

/home/brenda/Udacity/data-modeling-with-postgres/data/song_data/A/B/B/TRABBAM128F429D223.json


In [7]:
df = pd.read_json(filepath, lines = True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARBGXIG122988F409D,37.77916,-122.42005,California - SF,Steel Rain,SOOJPRH12A8C141995,Loaded Like A Gun,173.19138,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id','title','artist_id','year','duration']].values.tolist()[0]
song_data

['SOOJPRH12A8C141995', 'Loaded Like A Gun', 'ARBGXIG122988F409D', 0, 173.19138]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute('truncate dim_song;')
cur.execute(song_table_insert, song_data)
cur.execute('select * from dim_song')
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()
conn.commit()

('SOOJPRH12A8C141995', 'Loaded Like A Gun', 'ARBGXIG122988F409D', 0, Decimal('173.19138'))


In [10]:
#cur.close()
#conn.close()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()[0]
artist_data

['ARBGXIG122988F409D', 'Steel Rain', 'California - SF', 37.77916, -122.42005]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
#conn.rollback()
#cur.execute('truncate dim_artist;')
cur.execute(artist_table_insert, artist_data)
cur.execute('select * from dim_artist')
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()
conn.commit()

('ARBGXIG122988F409D', 'Steel Rain', 'California - SF', Decimal('37.77916'), Decimal('-122.42005'))


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files('data/log_data')

In [14]:
filepath = log_files[0]

In [15]:
df = pd.read_json(filepath, lines = True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
1,None,Logged In,Stefany,F,0,White,NaN,free,"Lubbock, TX",GET,Home,1540708070796,82,None,200,1541122176796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
df = df[df['page'] =='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Dance Gavin Dance,Logged In,Marina,F,0,Sutton,218.46159,free,"Salinas, CA",PUT,NextSong,1541064343796,47,Uneasy Hearts Weigh The Most,200,1541127957796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",48


In [17]:
df2 = df[df['artist'] == 'Weezer']
df2.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId


In [18]:
t = df['ts']
print(t)
#t.head()

0      1541121934796
2      1541122241796
3      1541122457796
4      1541126568796
5      1541127957796
           ...      
166    1541184339796
167    1541187675796
168    1541191023796
169    1541191251796
170    1541191397796
Name: ts, Length: 155, dtype: int64


In [19]:
t = df
t['ts'] = pd.to_datetime(t['ts'], unit = 'ms')
t.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,2018-11-02 01:25:34.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,2018-11-02 01:30:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,2018-11-02 01:34:17.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,2018-11-02 02:42:48.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Dance Gavin Dance,Logged In,Marina,F,0,Sutton,218.46159,free,"Salinas, CA",PUT,NextSong,1541064343796,47,Uneasy Hearts Weigh The Most,200,2018-11-02 03:05:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",48


- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [20]:
time_data = list([t['ts'], t['ts'].dt.hour, t['ts'].dt.day, t['ts'].dt.isocalendar().week, t['ts'].dt.month, t['ts'].dt.year, t['ts'].dt.weekday])
print('time_data data type: ', type(time_data))
print(time_data[3])
column_labels = (['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'])
print('colum_labels data type: ', type(column_labels))

time_data data type:  <class 'list'>
0      44
2      44
3      44
4      44
5      44
       ..
166    44
167    44
168    44
169    44
170    44
Name: week, Length: 155, dtype: UInt32
colum_labels data type:  <class 'list'>


In [21]:
time_df = dict(zip(column_labels,time_data))
print('time_df data type', type(time_df))
time_df = pd.DataFrame(time_df)
time_df.head()

time_df data type <class 'dict'>


,start_time,hour,day,week,month,year,weekday
0,2018-11-02 01:25:34.796,1,2,44,11,2018,4
2,2018-11-02 01:30:41.796,1,2,44,11,2018,4
3,2018-11-02 01:34:17.796,1,2,44,11,2018,4
4,2018-11-02 02:42:48.796,2,2,44,11,2018,4
5,2018-11-02 03:05:57.796,3,2,44,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
#conn.rollback()
for i, row in time_df.iterrows():
    #print(row)
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [23]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,2018-11-02 01:25:34.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,2018-11-02 01:30:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,2018-11-02 01:34:17.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,2018-11-02 02:42:48.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Dance Gavin Dance,Logged In,Marina,F,0,Sutton,218.46159,free,"Salinas, CA",PUT,NextSong,1541064343796,47,Uneasy Hearts Weigh The Most,200,2018-11-02 03:05:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",48


In [24]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,101,Jayden,Fox,M,free
2,83,Stefany,White,F,free
3,83,Stefany,White,F,free
4,66,Kevin,Arellano,M,free
5,48,Marina,Sutton,F,free


In [25]:
user_df = user_df.drop_duplicates()
user_df.head()

,userId,firstName,lastName,gender,level
0,101,Jayden,Fox,M,free
2,83,Stefany,White,F,free
4,66,Kevin,Arellano,M,free
5,48,Marina,Sutton,F,free
7,86,Aiden,Hess,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [26]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [27]:
for index, row in t.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    print("Song: %s - Artist: %s - Duration:  %s"%(row.song, row.artist, row.length))
    results = cur.fetchone()
    print(results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    
    #print(songid, artistid)

    #insert songplay record
    #songplay_table_insert = (""" insert into fac_songplay (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    songplay_data = (row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'], row['location'], row['userAgent'])
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

cur.execute('select count(*) from fac_songplay;')
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
conn.commit()

Song: Am I High (Feat. Malice) - Artist: N.E.R.D. FEATURING MALICE - Duration:  288.9922
None
Song: A Lack Of Color (Album Version) - Artist: Death Cab for Cutie - Duration:  216.42404
None
Song: I Have A Wish - Artist: Tracy Gang Pussy - Duration:  221.33506
None
Song: Monster (Album Version) - Artist: Skillet - Duration:  178.02404
None
Song: Uneasy Hearts Weigh The Most - Artist: Dance Gavin Dance - Duration:  218.46159
None
Song: Falta Te Dizer - Artist: Dalto - Duration:  190.40608
None
Song: Family Business - Artist: Kanye West - Duration:  278.88281
None
Song: Lucky (Album Version) - Artist: Jason Mraz & Colbie Caillat - Duration:  189.6224
None
Song: Journey To The Past  (LP Version) - Artist: Liz Callaway - Duration:  175.43791
None
Song: Everything I Try to Do_ Nothing Seems to Turn Out Right - Artist: The Decemberists - Duration:  242.59873
None
Song: Impossible - Artist: Christina Aguilera - Duration:  254.48444
None
Song: Secrets - Artist: OneRepublic - Duration:  224.6787

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [28]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.